In [6]:
import os
import sys
import pickle

PROJECT_ROOT = os.path.abspath('..')
sys.path.append(PROJECT_ROOT)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sleeprnn.helpers.reader import load_dataset
from sleeprnn.common import constants, viz, pkeys
from sleeprnn.data import utils, stamp_correction
from sleeprnn.detection.postprocessor import PostProcessor
from sleeprnn.detection import det_utils
from figs_thesis import fig_utils

viz.notebook_full_width()

RESULTS_PATH = os.path.join(PROJECT_ROOT, 'results')

In [11]:
class PredictedNSRR(object):
    def __init__(
        self,
        experiment_folder='20210716_from_20210529_thesis_indata_5cv_e1_n2_train_moda_ss_ensemble_to_e1_n2_train_nsrr_ss',
        grid_folder='v2_time',
        page_duration=30,
        min_separation=0.5,
        min_duration=0.5,
        max_duration=3.0,
    ):
        self.experiment_folder = experiment_folder
        self.grid_folder = grid_folder
        self.subject_to_fold_map = self._hash_predictions()
        self.all_ids = np.sort(list(self.subject_to_fold_map.keys()))
        
        post_params = {
            pkeys.PAGE_DURATION: page_duration,
            pkeys.SS_MIN_SEPARATION: min_separation,
            pkeys.SS_MIN_DURATION: min_duration,
            pkeys.SS_MAX_DURATION: max_duration,
        }
        self.postprocessor = PostProcessor(constants.SPINDLE, params=post_params)
    
    def get_fold_probabilities(self, fold_id):
        pred_path = os.path.join(RESULTS_PATH, 'predictions_nsrr_ss', self.experiment_folder, self.grid_folder)
        fold_path = os.path.join(pred_path, 'fold%d' % fold_id, 'prediction_n2_test.pkl')
        with open(fold_path, 'rb') as handle:
            proba_dict = pickle.load(handle)
        return proba_dict
    
    def get_subject_probabilities(self, subject_id):
        fold_id = self.subject_to_fold_map[subject_id]
        proba_dict = self.get_fold_probabilities(fold_id)
        subject_proba = proba_dict[subject_id]
        return subject_proba
    
    def get_subject_stamps(self, subject_id, subject_n2_pages=None, subject_probabilities=None, threshold=0.5):
        if subject_probabilities is None:
            subject_proba = self.get_subject_probabilities(subject_id)
        marks = self.postprocessor.proba2stamps(subject_proba, thr=threshold, pages_indices_subset=subject_n2_pages)
        return marks
    
    def get_fold_stamps(self, fold_id, fold_n2_pages_dict=None, threshold=0.5):
        proba_dict = self.get_fold_probabilities(fold_id)
        fold_subjects = list(proba_dict.keys())
        
        if fold_n2_pages_dict is None:
            fold_n2_pages_dict = {s: None for subject_id in fold_subjects}
        
        marks_dict = {}
        for subject_id in fold_subjects:
            subject_proba = proba_dict[subject_id]
            subject_marks = self.get_subject_stamps(
                subject_id, 
                subject_n2_pages=fold_n2_pages_dict[subject_id], 
                subject_probabilities=subject_proba, threshold=threshold)
            marks_dict[subject_id] = subject_marks
        return marks_dict, proba_dict
    
    def get_
    
    def _hash_predictions(self):
        pred_path = os.path.join(RESULTS_PATH, 'predictions_nsrr_ss', self.experiment_folder, self.grid_folder)
        folds = os.listdir(pred_path)
        folds = [int(f.split("fold")[-1]) for f in folds]
        folds = np.sort(folds)
        
        subject_to_fold_map = {}
        for fold_id in folds:
            proba_dict = self.get_fold_probabilities(fold_id)
            fold_subjects = list(proba_dict.keys())
            for subject_id in fold_subjects:
                subject_to_fold_map[subject_id] = fold_id
        return subject_to_fold_map

In [12]:
preds = PredictedNSRR()

In [16]:
nsrr = load_dataset(constants.NSRR_SS_NAME, load_checkpoint=True)

Dataset nsrr_ss with 3 patients.
Loading from checkpoint... Loaded
Global STD: None
Dataset nsrr_ss with 11593 patients.


# Reproducción de tendencias

In [ ]:
n_folds = 2


